## What is needed to make a Kamodo Reader:

We use the `swmfie_4Dcdf.py` and `gitm_4Dcdf.py` files as templates for constructing the TIEGCM Kamodo object (class)




**Model specific remedy functions**

1. Define the variables that make up the model output
2. Function to get hours since midnight from datetime string
3. Get datetime string in format "YYYY-MM-SS HH:mm:SS" from filename
4. Get datetime timestamp in UTC from datetime string

**Construct a MODEL class that inherits Kamodo**    ``class TIEGCM(Kamodo):``  
  
1. Initialize the class
 - define inputs
 - super().__init__() so that your class inherits any methods from Kamodo
 - add check to test if your files exist
 - establish the time attributes that will be used to identify files by their date
 - establish beginning time and end time of file list
 - add condition the exits code if there is only 1 file, because interpolator code will break
 - Store our inputs as class attributes to the class
 - Access the first file and GET a list of the variable names in this file set
     - This is merely a check to see if the requested variables are in the files
 - Store coordinate data as class attributes    
 - Store the requested variables into a dictionary as the variables attributes
     - this will contain units, dtype, and the data associated with the requested variable
     - close the netcdf file 
 - Register the interpolators for each variable
     - the swmfie datasets had only 3d variables, tiegcm has 4d mostly
     
2. Add a function to regsiter the variables to be interpolated when called
    - we will need to register 3d AND 4d variables
        
     
3. For the purposes of GEODYN, we ignore the plotting codes.... (sorry, everybody else)
 


### Lets do it for the TIEGCM model output

In [1]:
####   Vertically Only:

# interp_den = np.zeros(  (np.size(tiegcm['lon']), np.size(tiegcm['lat'])))
# for ilon,lonval in enumerate(tiegcm['lon']):
#     for ilat,latval in enumerate(tiegcm['lat']):
#         interp_den[ilon, ilat] =  np.exp( np.interp( alt*1e5  ,tiegcm['ZG'][ut_index,:,  ilat, ilon], np.log(tiegcm['DEN'][ut_index,:,  ilat, ilon]))  )



In [2]:
from os import path
import time as ti
import glob
import numpy as np
from netCDF4 import Dataset
from datetime import datetime, timezone
from kamodo import Kamodo

import sys
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/KamodoGroupCodeShare/readers')

import reader_plotutilities as RPlot
import reader_utilities as RU

# import kamodo.readers.reader_plotutilities as RPlot
# import kamodo.readers.reader_utilities as RU


"""
Created on Mon June 14, 2021.
@author: rringuette -- modified by Zach
"""

'''
This code reads the TIEGCM model output (AKA secondary history files).
Each sechist file consists of 1 day of data for the TIEGCM.

CHANGELOG (short-term):
    - added some specificity to the standard variable names in tiegcm_varnames
    - removed unnecessary code snippet that exits code if only given 1 file.
    - Expanded current state of code to be beyond only wanting density
        - uncommented register_3D_variable()
    - removed condition to force filenames into a list for looping.  
        -We only want to run kamodo object for a single file.
TO DO:
    - Convert from pressure level to height 
        - I believe this is accessed through the ZG variable if I understand what is being requested...  
        - If this is referring to an interpolation, then I thought this was handled in the interpolators

'''


### Make a dict of the possible variable names in TIEGCM
###       the intended format is:   "Output VarName":['Latex_Varname', 'Latex_Unit' ]

###  The constituent species are output in units of mass mixing ratio (mmr).
###  Denoted by \psi_i, mmr is the fractional contribution of
###    a species i to the total mass density \rho_{total}, and 
###    is calculated as \psi_i = \rho_i / \rho_{total}, where 
###    \rho_i is the mass density of a constituent species.
tiegcm_varnames={
                 ### 4D Variables, vertical coordinate on midpoint levels (lev)
                 "ZGMID"    : ["ZG_mid"     ,"cm" ],  #  geometric height- interpolated to the mid points
                 "TN"       : ["T_N"        ,"K"  ],  #  neutral temperature    
                 "O2"       : ["psi_{O_2}"  ,"mmr"],  #  molecular oxygen   
                 "O1"       : ["psi_{O_1}"  ,"mmr"],  #  atomic oxygen    
                 "N2"       : ["psi_{N_2}"  ,"mmr"],  #  molecular nitrogen
                 "HE"       : ["psi_{He}"   ,"mmr"],  #  helium     
                 "NO"       : ["psi_{NO}"   ,"mmr"],  #  nitric oxide  
                 "N2"       : ["psi_{N_{2}}","mmr"],  #  molecular nitrogen 
                 "N4S"      : ["psi_{N4S}"  ,"mmr"],  #  ? N4S ?
                 "N2N"      : ["n_{2,N}"    ,"mmr"],  #  molecular nitrogen-- N2N is an artifact of an older TIEGCM version
                 "TE"       : ["T_e"        ,"K"  ],  #  electron temperature
                 "TI"       : ["T_i"        ,"K"  ],  #  ion temperature
                 "O2P"      : ["O_2+"  ,"1/cm**3"],   #  O2+ ion
                 "OP"       : ["O+"    ,"1/cm**3"],   #  O+ ion
                 "CO2_COOL" : ["CO_{2}_{cool}","erg/g/s"],  #  CO2 cooling rates
                 "NO_COOL"  : ["NO_{cool}"    ,"erg/g/s"],  #  NO cooling rates
                 "UN"       : ["U_N"        ,"cm/s"], #  neutral zonal wind (+EAST)
                 "VN"       : ["V_N"        ,"cm/s"], #  neutral meridional wind (+NORTH)
                # 
                ### 4D Variables, vertical coordinate on interface levels (ilev)
                 "DEN"      : ["rho"  ,"g/cm**3"],    #  total neutral mass density  
                 "ZG"       : ["Z_G"  ,"cm"],         #  geometric height  
                 "Z"        : ["Z"    ,"cm"],         #  heopotential height (cm)  
                 "NE"       : ["N_e"  ,"1/cm**3"],    #  electron density
                 "OMEGA"    : ["omega","1/s"],        #  vertical motion
                 "POTEN"    : ["poten","V"],          #  electric potential
                #
                ### 4D Variables, vertical coordinate on interface mag levels (imlev)
                 "ZGMAG"  : ["ZG_{mag}",""],          #  Geopotential Height on Geomagnetic Grid (km)
                #
                ### 3D Variables,  (time, lat, lon)
                 "TEC"  : ["TEC"   ,"1/cm**2"],       #  Total Electron Content
                 "TLBC"  : ["TLBC" ,"K"],             #  Lower boundary condition for TN
                 "ULBC"  : ["ULBC" ,"cm/s"],          #  Lower boundary condition for UN
                 "VLBC"  : ["VLBC" ,"cm/s"],          #  Lower boundary condition for VN
                 "TLBC_NM"  : ["TLBC_NM",""],         #  Lower boundary condition for TN (TIME N-1)
                 "ULBC_NM"  : ["ULBC_NM",""],         #  Lower boundary condition for UN (TIME N-1)
                 "VLBC_NM"  : ["VLBC_NM",""],         #  Lower boundary condition for VN (TIME N-1)
                  }

#####--------------------------------------------------------------------------------------
##### Define some helpful functions for dealing with time systems
@np.vectorize
def dts_to_hrs(datetime_string, filedate):
    '''Get hours since midnight from datetime string'''
    
    return (datetime.strptime(datetime_string, '%Y-%m-%d %H:%M:%S').replace(tzinfo=timezone.utc)\
            -filedate).total_seconds()/3600.


def dts_to_ts(file_dts):
    '''Get datetime timestamp in UTC from datetime string'''    
    
    return datetime.timestamp(datetime.strptime(file_dts, '%Y-%j'
                                                ).replace(tzinfo=timezone.utc))


##### --------------------------------------------------------------------------------------
###   Construct a TIEGCM class that inherits Kamodo       
class TIEGCM(Kamodo): 
    def __init__(self, file_path, filename, variables_requested=[], runname="noname",
                 filetimes=False, verbose=False, gridded_int=False, printfiles=True,
                 **kwargs): 
        
        #### Use a super init so that your class inherits any methods from Kamodo
        super(TIEGCM, self).__init__()
        
                
#         if np.size(filenames) == 1:
#             filenames = [filenames]

        #### establish beginning time and end time of file list
        ####  
        self.datetimes = []
        year = file_path.split('/')[-2]
#         for i in filenames:
        print(filename)
        idoy  = filename.split('\\')[0][1:-3]
        print('idoy: ', idoy)
        string_date = year+'-'+idoy  #'YYYY-DDD'            
        self.datetimes.append(string_date)# return strings in format = YYYY-DDD
            
        self.timerange0={'min':self.datetimes[0], 'max':self.datetimes[-1], 'n':len(filename)}
        self.timerange = self.timerange0
        print(self.datetimes)
        self.filetimes=[dts_to_ts(file_dts) for file_dts in self.datetimes]   #timestamps in UTC     
        if filetimes: 
            return      
        
                
        #### Store our inputs as class attributes to the class
        self.filename      = filename
        self.runname       = runname
        self.missing_value = np.NAN
        self._registered   = 0
        self.variables     = dict()
        if printfiles: 
            print('Files:')
            for file in self.filename: print(file)

        #### Access the  file and get a list of the 
        ####     variable names in this file set
        #### This is merely a check to see if the requested
        ####     variables are in the files
        cdf_data = Dataset(file_path+filename, 'r')
        
        if len(variables_requested)>0:
            gvar_list = [key for key in cdf_data.variables.keys() if key \
                         in variables_requested]
            if len(gvar_list)!=len(variables_requested):
                err_list = [item for item in variables_requested if item not in \
                            cdf_data.variables.keys()]
                print('Some requested variables are not available:', err_list)
        else:
            gvar_list = [key for key in cdf_data.variables.keys() \
                         if key not in cdf_data.dimensions.keys() ]  
        
        #### Store coordinate data as class attributes    
        self._time = np.array(cdf_data.variables['time'])
        self._lev = np.array(cdf_data.variables['lev'])
        self._ilev = np.array(cdf_data.variables['ilev'])
        self._lat = np.array(cdf_data.variables['lat'])
        self._lon = np.array(cdf_data.variables['lon'])

        
        
        #### Store the requested variables into a dictionary 
        ####     as the variables attributes
        #### This will contain units, dtype, and the data
        self.variables = {key:{'units':cdf_data.variables[key].units, 'dtype':np.float32,
                               'data':np.array(cdf_data.variables[key])}\
                          for key in gvar_list}
        #### Close the CDF file
        cdf_data.close()
        if verbose: print(f'Took {ti.perf_counter()-t0:.6f}s to read in data')
            
            
        #### register interpolators for each  requested variable
        varname_list = [key for key in self.variables.keys()]  #store original list b/c gridded interpolators
        t_reg = ti.perf_counter()
        
        for varname in varname_list:
            if varname in ['TEC','TLBC','ULBC','VLBC',
                                'TLBC_NM','ULBC_NM','VLBC_NM']:
                self.register_3D_variable(self.variables[varname]['units'], 
                                      self.variables[varname]['data'], varname,
                                      gridded_int)
            else:
#                 print('4D', varname, self.variables[varname]['data'].shape)
#                 print(self.variables[varname]['units'])
                self.register_4D_variable(self.variables[varname]['units'], 
                                      self.variables[varname]['data'], varname,
                                      gridded_int)
                
    ##### Define and register a 3D variable -----------------------------------------
    def register_3D_variable(self, units, variable, varname, gridded_int):
        """Registers a 3d interpolator with 3d signature"""
        
        #define and register the interpolators
        xvec_dependencies = {'time':'s','lat':'deg','lon':'deg'}
        self = RU.regdef_3D_interpolators(self, units, variable, self._time, 
                                       self._lat, self._lon, varname, 
                                       xvec_dependencies, gridded_int)       
        return 
    
    
    #### Define and register a 4D variable -----------------------------------------
    def register_4D_variable(self, units, variable, varname, gridded_int):
        """Registers a 4d interpolator with 4d signature"""
    
        ilev_list = ['DEN', 'ZG', 'Z']                                                         # index with ilev
        lev_list  = ['TN', 'O2', 'O1', 'N2', 'HE', 'ZGMID', 'NO','CO2_COOL', 'NO_COOL', 'N2N'] # index with lev
             
        ####  Get the correct coordinates
        lat = getattr(self, '_lat')  
        lon = getattr(self, '_lon')
        
        if varname in ilev_list:
            vert_coord = getattr(self, '_ilev')
            str_vert = 'ilev'
        elif varname in lev_list:
            vert_coord = getattr(self, '_lev')
            str_vert = 'lev'

        #### define and register the interpolators
        xvec_dependencies = {'time':'s', str_vert:' ','lat':'deg','lon':'deg'}
        
        coord_str = 't,'+str_vert+'lat,lon'
        self = RU.regdef_4D_interpolators(self, units, variable, 
                                          self._time, vert_coord, lat, lon,
                                          varname, xvec_dependencies, gridded_int)
        return







/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/sympy/physics/units/quantities.py:77: SymPyDeprecationWarning: 

Moving method to UnitSystem class has been deprecated since SymPy 1.5.
Use unit_system.set_quantity_dimension or
nanotesla.set_global_relative_scale_factor instead. See
https://github.com/sympy/sympy/issues/17765 for more info.

  SymPyDeprecationWarning(
/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/sympy/physics/units/quantities.py:88: SymPyDeprecationWarning: 

Moving method to UnitSystem class has been deprecated since SymPy 1.5.
Use unit_system.set_quantity_scale_factor or
nanotesla.set_global_relative_scale_factor instead. See
https://github.com/sympy/sympy/issues/17765 for more info.

  SymPyDeprecationWarning(
/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/sympy/physics/units/quantities.py:77: SymPyDeprecationWarning: 

Moving method to UnitSystem class has been deprecated since SymPy 1.5.
Use unit_system.set_quantity_dimension or 

In [3]:
# import numpy as np
# # from kamodo.readers.swmfie_4Dcdf import SWMF_IE, swmfie_varnames
# from plotly.offline import init_notebook_mode, iplot, plot
# init_notebook_mode(connected = True)

In [4]:
# each TEIGCM model output (sechist) file has one days worth of data

# filename = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
# run_name = 's3'

file_path = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
filenames = 's320.nc'
run_name = 'zachs_data_from_cheyenne'



# # tiegcm = TIEGCM(file_path, filenames, runname=run_name, printfiles=False, gridded_int=True)  #load all variables, creates gridified interpolators
tiegcm = TIEGCM(file_path, filenames,variables_requested=['DEN', 'TN'], runname=run_name, printfiles=False, gridded_int=True)  #load all variables, creates gridified interpolators

# tiegcm

s320.nc
idoy:  320
['2003-320']


<ipython-input-2-3df2dab979dc>:183: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  'data':np.array(cdf_data.variables[key])}\


In [6]:
tiegcm

/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/sympy/core/relational.py:492: SymPyDeprecationWarning: 

Eq(expr) with rhs default to 0 has been deprecated since SymPy 1.5.
Use Eq(expr, 0) instead. See
https://github.com/sympy/sympy/issues/16587 for more info.

  SymPyDeprecationWarning(


TIEGCM([(TN(xvec),
         <function reader_utilities.define_4d_interpolator.<locals>.interpolator(xvec)>),
        (TN,
         <function reader_utilities.define_4d_interpolator.<locals>.interpolator(xvec)>),
        (TN_ijk(time, lev, lat, lon),
         <function wrapped(time=array([345660., 345720., 345780., 345840., 345900., 345960., 346020.,
       346080., 346140., 346200., 346260., 346320., 346380., 346440.,
       346500., 346560., 346620., 346680., 346740., 346800., 346860.,
       346920., 346980., 347040.]), lev=array([-6.75, -6.25, -5.75, -5.25, -4.75, -4.25, -3.75, -3.25, -2.75,
       -2.25, -1.75, -1.25, -0.75, -0.25,  0.25,  0.75,  1.25,  1.75,
        2.25,  2.75,  3.25,  3.75,  4.25,  4.75,  5.25,  5.75,  6.25,
        6.75,  7.25]), lat=array([-87.5, -82.5, -77.5, -72.5, -67.5, -62.5, -57.5, -52.5, -47.5,
       -42.5, -37.5, -32.5, -27.5, -22.5, -17.5, -12.5,  -7.5,  -2.5,
         2.5,   7.5,  12.5,  17.5,  22.5,  27.5,  32.5,  37.5,  42.5,
        47.5,  52.5, 

In [5]:
sys.exit(0)

SystemExit: 0

/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 30 15:21:50 2021
@author: rringuet +modified by zach to work with TIEGCM
Adapted for speed to execute for a single point 
"""
#coding imports
import numpy as np
import pandas as pd
import glob, sys
from datetime import datetime, timedelta, timezone
import time
import os
# from kamodo.readers.ctipe_faster_wrapped import CTIPe, ctipe_varnames 

#need input times to be timestamps since 1970 to pull data from the correct files
#modified to reduce number of times the ctipe reader is called to reduce calc time



def TIEGCMVariables():
    '''return a list of all the possible variables in TIEGCM'''
    
    return tiegcm_varnames




def ts_to_hrs(time_val, filedate):
    '''Convert array of timestamps to hours since midnight of filedate string'''
    
    file_datetime = datetime.strptime(filedate+' 00:00:00', '%Y-%m-%d %H:%M:%S')
    return (datetime.utcfromtimestamp(time_val)-file_datetime).total_seconds()/3600.





def hrs_to_ts(time_val, filedate):
    '''Convert array of hours since midnight of filedate string to timestamps'''
    
    file_datetime = datetime.strptime(filedate+' 00:00:00', '%Y-%m-%d %H:%M:%S').replace(tzinfo=timezone.utc)    
    return datetime.timestamp(file_datetime+timedelta(hours=time_val))





sample_ilev = np.linspace(-7, 7, 29,dtype=float)   #global variable
def CalcIlev(H, t, height, lat, lon):
    '''Approximate ilev by inverting the gridded height function CTIPe.H for one sat point'''
    
    rough_height = H(np.array([[t, ilev, lat, lon] for ilev in sample_ilev]))
    ilev_range = np.sort(sample_ilev[np.argsort(abs(height-rough_height))[0:2]])
    test_ilev = np.linspace(ilev_range[0],ilev_range[1],100,dtype=float)
    finer_height = H(np.array([[t, ilev, lat, lon] for ilev in test_ilev]))
    return test_ilev[np.argmin(abs(height-finer_height))]




    

def TIEGCM_Single_FlyAway(tiegcm, variable_list, sat_time, sat_height, sat_lat, sat_lon, 
                  z_dependence=['none'], dz=''):
    '''fly satellite through TEIGCM model data, per position
    sat_time, sat_height, sat_lat, and sat_lon are all floats, not arrays
    z_dependence = ["none","height","ilev"] if variables depend on all three options
       dependence must be in same order as variable_list to match with variable names'''
    
        
    #Create satellite tracks with appropriate inputs
    file_date = tiegcm.datetimes[0][0:10]
    model_sat_time = ts_to_hrs(sat_time, file_date)
    if 'H' in variable_list: variable_list.remove('H')  #H only needed if other functions require ilev
    sat_track = {}  #initialize list of satellite tracks
    if "none" in z_dependence:
        sat_track['none']=[model_sat_time,sat_lat,sat_lon]
    if "height" in z_dependence:  #if function requires height (in km)
        sat_track['height']=[model_sat_time,sat_height/1000.,sat_lat,sat_lon]
    if "ilev" in z_dependence:  #if ilev is required for at least one variable
        ##### FROM ZACH--- 
        #### CalcIlev was given wrong inputs: 
        ####           use model_sat_time   instead of   sat_time
        ####           use sat_height*1000. instead of   sat_height

        sat_ilev = CalcIlev(tiegcm.ZG, *[model_sat_time,sat_height*1000.,sat_lat,sat_lon])  
        sat_track['ilev']=[model_sat_time,sat_ilev,sat_lat,sat_lon]

    #retrieve interpolator and interpolate data for each variable. 
    results = {variable_list[i]: tiegcm[variable_list[i]](sat_track[z_dependence[i]])[0] for i
               in range(len(variable_list))}
    
    #determine vertical derivatives for each variable if requested
    if dz!='':
        for i in range(len(variable_list)):
            if dz[i] and z_dependence[i]!='none':  #if dz requested and variable has a vertical dependence
                #generate tracks with slightly larger and smaller heights
                if z_dependence[i]=='height':
                    #stay within CTIPe height (km) boundaries
                    sat_height_low = sat_height/1000.-100
                    if sat_height_low <= 140.: sat_height_low = 140.
                    sat_height_high = sat_height/1000.+100.
                    if sat_height_high>=2000.: sat_height_high = 2000.
                    dz_track = [[model_sat_time,sat_height_low,sat_lat,sat_lon],
                                [model_sat_time,sat_height_high,sat_lat,sat_lon]]
                if z_dependence[i]=='ilev':
                    #stay within CTIPe ilev boundaries
                    sat_ilev_low = sat_ilev-1.
                    if sat_ilev_low<=1.: sat_ilev_low = 1.
                    sat_ilev_high = sat_ilev+1.
                    if sat_ilev_high >= 15.: sat_ilev_high = 15.
                    dz_track =  [[model_sat_time,sat_ilev_low,sat_lat,sat_lon],
                                [model_sat_time,sat_ilev_high,sat_lat,sat_lon]]
                dz_result = ctipe[variable_list[i]](dz_track)  #returns two values
                results[variable_list[i]+'_dz'] = dz_result[1]-dz_result[0]
    return results

def find_singlefile(file_pattern, file_dir,  sat_time, reader, dt=450., verbose=False):
    '''Find file containing given single time. Adjust if within dt seconds.'''
    
    ##### build lookup table
#     files, times, ts_list, filename = glob.glob(file_pattern), {}, [], ''
#     print('files', files)
#     print('times', times)
#     print('ts_list', ts_list)
#     print('filename', filename)
    files = file_pattern
    print('sattime:',sat_time )
    print('dt:', )
#     for f in files:
#         print('f:', f)
#         k = reader(file_dir, f, variables_requested=[], filetimes=True)
#         file_date = k.datetimes[0][0:10]
# #         print(file_date)

#         times[file_date] = [f,k.timerange['min'],k.timerange['max'],
#                             k.filetimes[0], k.filetimes[1]]
#         ts_list.extend([k.filetimes[0], k.filetimes[1]])
    
#     #choose file time is in
#     for file_date in times.keys():
#         if ((sat_time>=times[file_date][3]) & (sat_time<=times[file_date][4])):
#             filename = times[file_date][0]
#             break
#     if filename=='':  #if not in a file, try closest file if within dt
#         sat_time = [ts_list[abs(np.array(ts_list)-sat_time).argmin()] if \
#                           (abs(np.array(ts_list)-sat_time).min() < dt) else 0][0]
#         if sat_time==0:  #error if not within dt
#             raise AttributeError('Files in dir do not contain the given time.')
#         elif verbose:
#             print(f'Adjusting time to nearest file with {dt} seconds.')
#         for file_date in times.keys():  #otherwise, find file
#             if ((sat_time>=times[file_date][3]) & (sat_time<=times[file_date][4])):
#                 filename = times[file_date][0]
#                 break 
            
    return filename, sat_time

# if __name__=='__main__':
#     ''' Begin program '''
   
#     '''#initialize input parameters (filename, variable_name)
#     file_dir = 'C:/Users/rringuet/Kamodo_WinDev1/CTIPe/'
#     variable_list = ['rho']  #Test ilev with N_n, without ilev with T_e, 3D with TEC
#     #variable_list = ['rho', 'T', 'T_e', 'T_i', 'H', 'Vn_lat', 'Vn_lon', 'Vn_H', 
#     #                 'T_n', 'Rmt', 'N_e', 'N_n', 'Q_Solar', 'Q_Joule', 'Q_radiation', 
#     #                 'N_O', 'N_O2', 'N_N2', 'N_NO', 'N_NOplus', 'N_N2plus', 'N_O2plus', 
#     #                 'N_Nplus', 'N_Oplus', 'N_Hplus', 'sigma_P', 'sigma_H', 'Vi_lon', 
#     #                 'Vi_lat', 'W_Joule', 'Eflux_precip', 'Eavg_precip', 'TEC', 
#     #                 'E_theta140km', 'E_lambda140km', 'E_theta300km', 'E_lambda300km']
    
#     #generate iterator for sat_time testing, first position, and begin timing
#     n, ctipe = 80640, []  #80640
#     sat_time_arr = np.linspace(1426638400.0, 1426638400.0+80640.*2.5, n)  
#     sat_lat, sat_lon, sat_height = -20., 120., 400000.  #height in m 
    
    
#     #determine vertical variable dependency for each variable in variable_list
#     z_dependence = CTIPe_Single_Prerun(file_dir, variable_list)
#     print(f'Vertical dependence determined: {ti.perf_counter()-tic:.6f}')
#     #returns list of string(s) indicating vertical dependency of each variable   
#     '''
    
#     #collect arguments from the command line
#     #example: python C:\Users\rringuet\Kamodo_WinDev1\Kamodo-master\kamodo\readers\CTIPe_wrapper_fortrancmd.py
#     #       C:/Users/rringuet/Kamodo_WinDev1/CTIPe/ rho ilev 1426637500.0 400.0 -25.0 10.0
#     #syntax: file_dir, variable, z_dependence, sat_time, sat_height, sat_lat, sat_lon
#     file_dir = str(sys.argv[1]) #'C:/Users/rringuet/Kamodo_WinDev1/CTIPe/'
#     variable_list = ['DEN', 'ZG']# #['rho']
#     z_dependence = [sys.argv[3]] #['ilev']
    
#     ### ------Zach modified this to take a YYMMDDHHMMSS 
#     ###       since calculating UNIX time in GEODYN was a nightmare
#     YYMMDD = str(sys.argv[4])
#     HHMMSS = str(sys.argv[5])
#     sat_time_dt = pd.to_datetime(YYMMDD+HHMMSS , format='%y%m%d%H%M%S')
#     sat_time = time.mktime(sat_time_dt.timetuple())  
# #     print('sat_time (dt)  ',sat_time_dt)
# #     print('sat_time (unix)',sat_time )
#     ### -------End Zach's edit
    
#     sat_height = float(sys.argv[6]) #400. (in km)
#     sat_lat = float(sys.argv[7]) #-25.
#     sat_lon = float(sys.argv[8]) #90.
# #     print(file_dir, variable_list, z_dependence, sat_time, sat_height, sat_lat, sat_lon)

    
#     #find file nearest sat_time (within 450 seconds), correct sat_time if needed
#     filename, sat_time = find_singlefile(file_dir+'*nc', 
#                                          sat_time, TIEGCM, dt=450., verbose=False)
    
#     #get ctipe object for requested variable (+H too)
#     tiegcm = TIEGCM(filename, variables_requested=variable_list, printfiles=False)
       
# #     print(ctipe)
#     #get results requested for single position given
#     results_dict = TIEGCM_Single_FlyAway(tiegcm, variable_list, 
#                                     sat_time, sat_height, sat_lat, sat_lon, 
#                                     z_dependence=z_dependence, 
#                                     dz=[1])
#     print(results_dict)
        
# #     file = open(file_dir+'results.txt', 'w')
# #     file.write(f"{results_dict[variable_list[0]]:.15f}\n{results_dict[variable_list[0]+'_dz']:.15f}")
# #     file.close()
    
     

In [ ]:
file_path = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
filenames = ['s320.nc','s321.nc','s322.nc','s323.nc','s324.nc']



file_dir = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
variable_list = ['DEN', 'ZG']# #['rho']
z_dependence = ['ilev']



### ------Zach modified this to take a YYMMDDHHMMSS 
###       since calculating UNIX time in GEODYN was a nightmare
YYMMDD =  '031116' #str(sys.argv[4])
HHMMSS =   '121010' #str(sys.argv[5])
sat_time_dt = pd.to_datetime(YYMMDD+HHMMSS , format='%y%m%d%H%M%S')
sat_time = time.mktime(sat_time_dt.timetuple())  
#     print('sat_time (dt)  ',sat_time_dt)
#     print('sat_time (unix)',sat_time )
### -------End Zach's edit





sat_height = 470. #float(sys.argv[6]) #400. (in km)
sat_lat =  -25.   #float(sys.argv[7]) #-25.
sat_lon =  90.    #float(sys.argv[8]) #90.
#     print(file_dir, variable_list, z_dependence, sat_time, sat_height, sat_lat, sat_lon)


#find file nearest sat_time (within 450 seconds), correct sat_time if needed
filename, sat_time = find_singlefile(filenames, file_dir,
                                     sat_time, TIEGCM, dt=450., verbose=False)

#get ctipe object for requested variable (+H too)
tiegcm = TIEGCM(filename, variables_requested=variable_list, printfiles=False)

#     print(ctipe)
#get results requested for single position given
results_dict = TIEGCM_Single_FlyAway(tiegcm, variable_list, 
                                sat_time, sat_height, sat_lat, sat_lon, 
                                z_dependence=z_dependence, 
                                dz=[1])
print(results_dict)




In [ ]:
sat_time
file_datetime = datetime.strptime(filedate+' 00:00:00', '%Y-%m-%d %H:%M:%S')

In [ ]:
file_dates = []

filenames = ['s320.nc','s321.nc','s322.nc','s323.nc','s324.nc']
year = file_path.strip('//')[-4:]
for file in filenames:
    idoy = file[1:4]
    file_dates.append( pd.to_datetime(year+'-'+idoy, format='%Y-%j'))

sat_time_dt = (datetime.utcfromtimestamp(sat_time))

from datetime import datetime,timedelta
rounded_sat_time = sat_time_dt - timedelta(hours = sat_time_dt.hour % 24)
rounded_sat_time


In [ ]:
rounded_sat_time

In [ ]:
def nearest(file_dates, sat_time_dt):
    return min(file_dates, key=lambda x: abs(x - sat_time_dt))




In [ ]:
print(file_dates)
print(sat_time_dt)
print(rounded_sat_time)
print('want file', nearest(file_dates, rounded_sat_time))



In [ ]:
# # -*- coding: utf-8 -*-
# """
# Created on Tue Mar 30 15:21:50 2021

# @author: rringuet
# """
# #coding imports
# import glob
# import numpy as np
# # from kamodo.readers.ctipe_4D import TIEGCM, tiegcm_varnames
# # import satelliteflythrough.wrapper_utilities as U
 
# import sys
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/KamodoGroupCodeShare/satelliteflythrough/')
# import wrapper_utilities as U

    
# #need input times to be timestamps since 1970 to pull data from the correct files
   
# def TIEGCMVariables():
#     '''return a list of all the possible variables in TIEGCM'''
    
#     return tiegcm_varnames

# def FlyAway(filename, variable_list, sat_time, sat_height, sat_lat, sat_lon, 
#                   plot_sampling=4000, plot_file='', verbose=False):
#     '''fly satellite through TIEGCM model data, per file'''
    
#     #Check that sat data is all the same length, will error if not
#     U.sat_data_check(sat_time, sat_height, sat_lat, sat_lon)
    
#     #create ctipe kamodo object, initialize some variables
# #     ctipe = CTIPe(filename, variables_requested=variable_list, printfiles=True, 
# #                   gridded_int=False)
#     tiegcm = TIEGCM(file_path, filenames,
#                     variables_requested=['DEN'], 
#                     printfiles=False, 
#                     gridded_int=False)  #load all variables, creates gridified interpolators

# #     if 'H' in variable_list: variable_list.remove('H')  #H only needed if other functions require ilev
    
#     #create satellite tracks and interpolate data for each variable
#     results = U.Generic_FlyAway(tiegcm, variable_list, sat_time, sat_height, sat_lat,
#                                 sat_lon, plot_file, plot_sampling, verbose=False)
        
#     if verbose: print(f'Done for {filename}\n')
#     return results

# def TIEGCM_SatelliteFlythrough(file_dir, variable_list, sat_time, sat_height, sat_lat, 
#                               sat_lon, dt=450., plots=False, daily_plots=False, plot_close=True, 
#                               plot_sampling=4000, verbose=False):
#     '''
#     Execute flythrough for TIEGCM model data. Returns results_dict, results_units.
#     results_dict is a dictionary of the interpolated data for the entire data set
#         sorted by variable name.
#     results_units is a dictionary of the units from the model for each variable.
#     file_dir is a string indicating where the data files are located.
#     variable_list is a list of strings of the desired variables. 
#     sat_time is an array of timestamp values.
#     sat_height is an array of heights in meters.
#     sat_lat and sat_lon ar arrays of latitudes and longitudes in degrees.
#     Set plots=True to get plots of the entire data set for each variable.
#     Set daily_plots=True to get plots of the data set for each variable and each file.
#     Set plot_close=False to keep plots of the entire data set open.
#     '''
    
#     file_dir = '/data/data_geodyn/atmos_models_data/tiegcm/2003/'
#     files = glob.glob(file_dir+'*.nc')  #look for wrapped and original data
#     file_patterns = np.unique([file_dir+''+f.split('/')[-1].split('\\')[-1][:]+\
#                                '' for f in files]) 

#     varlist_3d = [''] #add 3D items to list as discovered ----------
#     results_dict, results_units = U.generic_SatelliteFlythrough(file_dir, file_patterns, 
#                                 sat_time, TIEGCM, FlyAway, variable_list, sat_height, 
#                                 sat_lat, sat_lon, daily_plots, plot_sampling, 
#                                 tiegcm_varnames, varlist_3d, plots, plot_close, 
#                                 dt=dt, verbose=verbose)
#     return results_dict, results_units

# if __name__=='__main__':
#     ''' Begin program '''
#     #initialize input parameters (filename, variable_name)
#     file_dir = 'C:/Users/rringuet/Kamodo_WinDev1/CTIPe/'
#     variable_list = ['T_e', 'TEC']  #Test ilev with N_n, without ilev with T_e, 3D with TEC
#     #variable_list = ['rho', 'T', 'T_e', 'T_i', 'H', 'Vn_lat', 'Vn_lon', 'Vn_H', 
#     #                 'T_n', 'Rmt', 'N_e', 'N_n', 'Q_Solar', 'Q_Joule', 'Q_radiation', 
#     #                 'N_O', 'N_O2', 'N_N2', 'N_NO', 'N_NOplus', 'N_N2plus', 'N_O2plus', 
#     #                 'N_Nplus', 'N_Oplus', 'N_Hplus', 'sigma_P', 'sigma_H', 'Vi_lon', 
#     #                 'Vi_lat', 'W_Joule', 'Eflux_precip', 'Eavg_precip', 'TEC', 
#     #                 'E_theta140km', 'E_lambda140km', 'E_theta300km', 'E_lambda300km']
    
#     #generate a fake satellite track    
#     from kamodo.satelliteflythrough.SatelliteFlythrough import SampleTrajectory as ST
#     traj_dict = ST(1426660000.0-30000., 1426791280.0)
    
#     results_dict, results_units = CTIPe_SatelliteFlythrough(file_dir, variable_list, 
#                                     traj_dict['sat_time'], traj_dict['sat_height'], 
#                                     traj_dict['sat_lat'], traj_dict['sat_lon'], dt=450.,
#                                     plots=True, daily_plots=False, plot_close=False,
#                                     verbose=False)

In [ ]:
# file_patterns